In [3]:
import datasets
import pandas as pd
from datasets import Dataset, load_dataset
from transformers import BertTokenizer

max_input_length = 512
max_target_length = 128
 
lcsts_part_1=pd.read_table('../../../resources/corpus/LCSTS_ORIGIN/DATA/PART_II.txt', header=None,
                           warn_bad_lines=True, error_bad_lines=False, sep='<[/d|/s|do|su|sh][^a].*>', encoding='utf-8') # PART I  too big, use PART II instead
lcsts_part_1=lcsts_part_1[0].dropna()
lcsts_part_1=lcsts_part_1.reset_index(drop=True)
lcsts_part_1=pd.concat([lcsts_part_1[1::2].reset_index(drop=True), lcsts_part_1[::2].reset_index(drop=True)], axis=1)
lcsts_part_1.columns=['document', 'summary']
 
lcsts_part_2=pd.read_table('../../../resources/corpus/LCSTS_ORIGIN/DATA/PART_II.txt', header=None,
                           warn_bad_lines=True, error_bad_lines=False, sep='<[/d|/s|do|su|sh][^a].*>', encoding='utf-8')
lcsts_part_2=lcsts_part_2[0].dropna()
lcsts_part_2=lcsts_part_2.reset_index(drop=True)
lcsts_part_2=pd.concat([lcsts_part_2[1::2].reset_index(drop=True), lcsts_part_2[::2].reset_index(drop=True)], axis=1)
lcsts_part_2.columns=['document', 'summary']
 
dataset_train = Dataset.from_dict(lcsts_part_1)
dataset_valid = Dataset.from_dict(lcsts_part_2)



C:\Users\Administrator\AppData\Local\Temp\ipykernel_8276\176903661.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  lcsts_part_1=pd.read_table('../../../resources/corpus/LCSTS_ORIGIN/LCSTS_ORIGIN/DATA/PART_II.txt', header=None,
C:\Users\Administrator\AppData\Local\Temp\ipykernel_8276\176903661.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  lcsts_part_1=pd.read_table('../../../resources/corpus/LCSTS_ORIGIN/LCSTS_ORIGIN/DATA/PART_II.txt', header=None,
C:\Users\Administrator\AppData\Local\Temp\ipykernel_8276\176903661.py:9: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  lcsts_part_1=pd.read_table('../../.

In [5]:
TokenModel = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(TokenModel)

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

In [6]:

def preprocess_function(examples):
    inputs = [str(doc) for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    inputs = [str(doc) for doc in examples["summary"]]
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(inputs, max_length=max_target_length, truncation=True)
 
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
tokenized_datasets_t = dataset_train.map(preprocess_function, batched=True)
tokenized_datasets_v = dataset_valid.map(preprocess_function, batched=True)
 
tokenized_datasets = datasets.DatasetDict({"train":tokenized_datasets_t,"validation": tokenized_datasets_v})
print(tokenized_datasets)

print(tokenized_datasets['train'][:10])

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10666
    })
    validation: Dataset({
        features: ['document', 'summary', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10666
    })
})
{'document': ['日前，方舟子发文直指林志颖旗下爱碧丽推销假保健品，引起哗然。调查发现，爱碧丽没有自己的生产加工厂。其胶原蛋白饮品无核心研发，全部代工生产。号称有“逆生长”功效的爱碧丽“梦幻奇迹限量组”售价高达1080元，实际成本仅为每瓶4元！', '韩方应对路径可以概括为：企业道歉担责；政府公正不护短；民间祈福关怀。他们深知形象的重要，竭力呵护企业品牌和国家形象。正如有评论，韩国“政府+企业+民众”三位一体式呵护韩国国家形象的“苦心经营”，的确有值得我们借鉴之处。', '63岁退休教师谢淑华，拉着人力板车，历时1年，走了2万4千里路，带着年过九旬的妈妈环游中国，完成了妈妈“一辈子在锅台边转，也想出去走走”的心愿。她说：“妈妈愿意出去走走，我就愿意拉着，孝心不能等，能走多远就走多远。', '昨天，包括工农中建交五大行在内的多家银行，不约而同地在官网发布公告称，它们的房地产贷款政策没有变化。多家银行表示，会支持居民购买首套住房。一名金融问题专家称，“目前房价不具备大涨大跌的基础，特别是一二线城市狂跌的可能性小。”', '广东4名律师致函中国民航局，要求其规定，航班起飞前要向乘客公布机组人员信息包括安全飞行时间、职业经历等。沪上业内人士认为，一般能被安排执飞任务的飞行员，均拥有民航认可的飞行资质。而责任心和使命感是无法通过飞行时间反映。', '任教五十年，钱理群在2012年教师节前夕宣布“告别教育”。从北大退休后，钱理群投身中学教育，试图“改变人心”，他以鲁迅自励，要在绝望中反抗，但基础教育十年试水，却令他收获